### ENCRYPTION DATABASE WITH AES-GCM

In [1]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import json

In [2]:
# Ganti 'nama_file.csv' dengan nama file CSV yang ingin Anda baca
file_path = "your csv directory"

# Membaca file CSV ke dalam dataframe
df = pd.read_csv(file_path)

In [11]:
df = df.head(10)

In [12]:
df

,Jan,Feb,Mar,Apr,Mei,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year Total
0,"377,045,713","379,113,097","372,316,090","392,567,778","398,734,898","411,232,840","368,853,507","415,321,833","382,962,945","467,893,711","375,815,944","424,064,387","4,765,922,743"
1,"242,370,197","212,251,985","204,414,196","233,523,016","220,730,925","233,194,586","221,971,536","246,355,021","236,335,091","292,120,864","201,786,738","243,465,095","2,788,519,250"
2,"576,420,037","528,756,483","546,400,555","617,218,714","587,424,126","547,152,791","550,558,762","555,374,056","478,131,369","580,774,960","418,898,225","533,925,422","6,521,035,500"
3,"262,642,225","247,718,521","281,930,873","330,248,572","274,214,556","292,985,200","281,301,030","302,722,271","272,821,443","337,484,741","213,824,938","270,660,844","3,368,555,214"
4,"283,962,745","273,753,242","275,621,256","275,377,925","285,094,002","271,323,533","266,368,767","278,199,918","274,482,338","340,876,987","272,031,107","319,181,859","3,416,273,679"
5,"264,618,010","262,560,291","266,737,078","291,233,128","266,804,240","265,814,874","262,475,102","327,169,088","279,381,522","341,443,345","228,436,701","285,087,305","3,341,760,684"
6,"217,605,126","202,406,935","205,281,943","251,661,179","205,936,466","222,309,023","228,219,727","216,370,586","205,156,152","248,363,183","155,247,937","172,204,443","2,530,762,700"
7,"250,304,747","220,955,617","239,413,809","279,512,633","223,743,139","258,200,971","240,139,882","271,225,530","231,088,815","313,723,510","218,526,485","287,813,635","3,034,648,773"
8,"255,194,364","233,694,343","245,456,148","280,149,467","233,751,924","265,984,894","271,807,000","271,945,498","240,868,948","287,434,527","181,170,109","230,451,812","2,997,909,034"
9,"480,110,487","449,252,634","441,566,370","392,514,687","476,342,526","505,133,298","442,169,670","477,236,876","458,940,797","520,116,162","414,042,405","498,157,979","5,555,583,891"


In [13]:
# Fungsi untuk membuat kunci acak
def generate_key():
    return AESGCM.generate_key(bit_length=128)  # Bit_length 128 sesuai dengan AES-GCM 128-bit

# Fungsi untuk enkripsi data
def encrypt_data(data, key, nonce):
    cipher = AESGCM(key)
    ciphertext = cipher.encrypt(nonce, str(data).encode(), None)
    return ciphertext

# Fungsi untuk enkripsi seluruh kolom dalam dataframe
def encrypt_dataframe_columns(df, key, nonce):
    encrypted_df = df.copy()
    for column in df.columns:
        encrypted_df[column] = df[column].apply(lambda x: encrypt_data(x, key, nonce))
    return encrypted_df

# Fungsi untuk dekripsi data
def decrypt_data(encrypted_data, key, nonce):
    cipher = AESGCM(key)
    decrypted_data = cipher.decrypt(nonce, encrypted_data, None)
    return decrypted_data.decode()

# Fungsi untuk memuat kunci enkripsi dari file JSON
def load_encryption_key(file_path):
    with open(file_path, 'r') as file:
        key_data = json.load(file)
        return bytes.fromhex(key_data['encryption_key'])
    
# Fungsi untuk menyimpan kunci enkripsi dan nonce ke dalam file JSON
def save_encryption_info(file_path, encryption_key, nonce):
    key_data = {'encryption_key': encryption_key.hex(), 'nonce': nonce.hex()}
    with open(file_path, 'w') as file:
        json.dump(key_data, file)

# Fungsi untuk memuat kunci enkripsi dan nonce dari file JSON
def load_encryption_info(file_path):
    with open(file_path, 'r') as file:
        key_data = json.load(file)
        encryption_key = bytes.fromhex(key_data['encryption_key'])
        nonce = bytes.fromhex(key_data['nonce'])
        return encryption_key, nonce

In [14]:
# Generate kunci
encryption_key = generate_key()

# Generate nonce sekali saja
nonce = generate_key()

# Enkripsi semua kolom dalam dataframe
encrypted_df_sales = encrypt_dataframe_columns(df, encryption_key, nonce)

# Simpan kunci enkripsi dan nonce ke dalam file JSON
save_encryption_info('encryption_info.json', encryption_key, nonce)

# Memuat kunci enkripsi dan nonce dari file JSON
loaded_encryption_key, loaded_nonce = load_encryption_info('encryption_info.json')

In [15]:
# Dekripsi dataframe yang telah dienkripsi
decrypted_df_sales = encrypted_df_sales.copy()
for column in encrypted_df_sales.columns:
    decrypted_df_sales[column] = encrypted_df_sales[column].apply(
        lambda x: decrypt_data(x, loaded_encryption_key, loaded_nonce)
    )

In [16]:
encrypted_df_sales

,Jan,Feb,Mar,Apr,Mei,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year Total
0,b'\x80\xea\xe3M\xde\x8d_1\x87\xcb\r=\xe6\xab\x...,b'\x80\xea\xedM\xdf\x88Y1\x80\xc3\t\x06{\x8f\x...,b'\x80\xea\xe6M\xdd\x88\\1\x80\xc3\x0e%\xea\x9...,b'\x80\xe4\xe6M\xdb\x8f]1\x87\xcd\x06\xf2n?\x8...,b'\x80\xe4\xecM\xd9\x8a^1\x88\xc3\x06\xec\x95t...,b'\x87\xec\xe5M\xdc\x8aX1\x88\xce\x0e2)@\xef\x...,"b""\x80\xeb\xecM\xd6\x8cY1\x85\xca\t'\x94\xa3i'...","b""\x87\xec\xe1M\xdd\x8b[1\x88\xc9\r\x85\x1a\xb...","b""\x80\xe5\xe6M\xd7\x8fX1\x89\xce\x0bwx\x08r\x...",b'\x87\xeb\xe3M\xd6\x80Y1\x87\xcb\x0fQV\xb8s\x...,b'\x80\xea\xe1M\xd6\x88_1\x89\xce\n\\\xfaa\xd7...,"b""\x87\xef\xe0M\xde\x8f^1\x83\xc2\t\xa3K\xcc7\...",b'\x87\xf1\xe3W\xdb\x95S/\x82\xd6\t\xa3|8\xced...
1,"b'\x81\xe9\xe6M\xdd\x8eZ1\x81\xc3\t\xe27,\x8b\...",b'\x81\xec\xe6M\xdc\x8c[1\x89\xc2\x0b#\x9f\x04...,b'\x81\xed\xe0M\xda\x88^1\x81\xc3\x08\x87\x91U...,b'\x81\xee\xe7M\xdb\x8bY1\x80\xcb\x08CA\xca\xb...,"b""\x81\xef\xe4M\xd9\x8aZ1\x89\xc8\x0b\\nc0\x8a...",b'\x81\xee\xe7M\xdf\x80^1\x85\xc2\x08\x9cE\n\x...,b'\x81\xef\xe5M\xd7\x8e[1\x85\xc9\x08p\\\xef\x...,b'\x81\xe9\xe2M\xdd\x8c_1\x80\xc8\x0f;\xfd\x0b...,b'\x81\xee\xe2M\xdd\x8a_1\x80\xc3\x0f\n\xcf\xd...,b'\x81\xe4\xe6M\xdf\x8bZ1\x88\xcc\n\xd40\x11\x...,b'\x81\xed\xe5M\xd9\x81\\1\x87\xc9\x06\x85\x90...,b'\x81\xe9\xe7M\xda\x8f_1\x80\xc3\x0b\x1a\xc0\...,"b'\x81\xf1\xe3Y\xd6\x95_,\x89\xd6\x0c\xa2\x7f\..."
2,b'\x86\xea\xe2M\xda\x8bZ1\x80\xc9\tt\x1bJ\x95\...,b'\x86\xef\xecM\xd9\x8c\\1\x84\xc2\r9\xe4_\x02...,b'\x86\xe9\xe2M\xda\x89Z1\x85\xcf\x0b]\xd3\x85...,b'\x85\xec\xe3M\xdc\x88R1\x87\xcb\n\r\xf9i0\xb...,b'\x86\xe5\xe3M\xda\x8b^1\x81\xc8\x08\n\xfdd\x...,b'\x86\xe9\xe3M\xdf\x8cX1\x87\xc3\x0f\xbf\x1c\...,b'\x86\xe8\xe4M\xdb\x8cR1\x87\xcc\x0cV\xf3hP\x...,"b'\x86\xe8\xe1M\xdd\x8e^1\x80\xcf\x08#""@\x95\x...",b'\x87\xea\xecM\xdf\x8a[1\x83\xcc\x07\x0b*t}\x...,"b'\x86\xe5\xe4M\xd9\x8e^1\x89\xcc\x0e\x1dbL""\x...",b'\x87\xec\xecM\xd6\x80R1\x82\xc8\x0b\x81\xfb[...,b'\x86\xee\xe7M\xd7\x8b_1\x84\xc8\x0c\x17R\x85...,b'\x85\xf1\xe1S\xdf\x95Z.\x85\xd6\x0b\xa7\x7f\...
3,b'\x81\xeb\xe6M\xd8\x8dX1\x82\xc8\x0b(\xba\x16...,b'\x81\xe9\xe3M\xd9\x88R1\x85\xc8\x0f\xd8L\x16...,b'\x81\xe5\xe5M\xd7\x8aZ1\x88\xcd\rl\xe3\xb3\x...,b'\x80\xee\xe4M\xdc\x8dR1\x85\xcd\x0c\x1c\x98L...,b'\x81\xea\xe0M\xdc\x88^1\x85\xcf\x08\x94\xe12...,b'\x81\xe4\xe6M\xd7\x81_1\x82\xca\x0ee\x08\xad...,b'\x81\xe5\xe5M\xdd\x89[1\x80\xc9\x0eQYX\xe8!i...,b'\x80\xed\xe6M\xd9\x8bX1\x82\xcd\x0f\x05\x86M...,b'\x81\xea\xe6M\xd6\x8b[1\x84\xce\rv\xf0\xe8\x...,b'\x80\xee\xe3M\xda\x81^1\x87\xce\x0f\xff\xd4[...,b'\x81\xec\xe7M\xd6\x8b^1\x89\xc9\x06J\xbe\xa5...,b'\x81\xea\xe4M\xd8\x8fZ1\x88\xce\nF\xc7x;\xf8...,"b'\x80\xf1\xe7W\xd6\x95_(\x85\xd6\x0c\xa6{,\xc..."
4,"b'\x81\xe5\xe7M\xd7\x8fX1\x87\xce\x0b\xcc""\x8f...",b'\x81\xea\xe7M\xd9\x8cY1\x82\xce\x0c\x1b\x0f\...,b'\x81\xea\xe1M\xd8\x8b[1\x82\xcf\x08\r\x88`\x...,b'\x81\xea\xe1M\xdd\x8e]1\x89\xc8\x0b\x07\xc2N...,b'\x81\xe5\xe1M\xde\x80^1\x80\xca\x0cQ\xa2\xfa...,b'\x81\xea\xe5M\xdd\x8bY1\x85\xc9\rD\x9a\xa2\x...,"b""\x81\xeb\xe2M\xdd\x8fR1\x87\xcc\txJ\xab\xecr...",b'\x81\xea\xecM\xdf\x80S1\x89\xcb\x06H\x91\x19...,b'\x81\xea\xe0M\xda\x81X1\x83\xc9\x06\xd6cI\xa...,b'\x80\xe9\xe4M\xd6\x8e\\1\x89\xc2\t\x93\x13+\...,b'\x81\xea\xe6M\xde\x8a[1\x81\xca\tm\xa3>\x97\...,b'\x80\xec\xedM\xdf\x81[1\x88\xcf\x07]P\xa8A\x...,b'\x80\xf1\xe0P\xd8\x95X*\x83\xd6\x08\xa0v\xc1...
5,"b'\x81\xeb\xe0M\xd8\x88R1\x80\xcb\x0e""v\x1d\xc...",b'\x81\xeb\xe6M\xdb\x8fZ1\x82\xc3\x0f\xff\x97\...,b'\x81\xeb\xe2M\xd9\x8a]1\x80\xcd\x06l\x02mJ4\...,b'\x81\xe4\xe5M\xdc\x8aY1\x81\xc8\x06\xf7\xa3\...,b'\x81\xeb\xe2M\xd6\x89^1\x82\xce\x0e!G\x14\x8...,b'\x81\xeb\xe1M\xd6\x88^1\x88\xcd\n\xea\xb3.\x...,b'\x81\xeb\xe6M\xda\x8e_1\x81\xca\x0cT\x07a\xa...,b'\x80\xef\xe3M\xdf\x8fS1\x80\xc2\x06f\r\xa6\x...,b'\x81\xea\xedM\xdd\x81[1\x85\xc8\x0cbs\xf1\xc...,b'\x80\xe9\xe5M\xda\x8dY1\x83\xce\x0bqz\x13[\x...,b'\x81\xef\xecM\xda\x8a\\1\x87\xca\x0f\\\xbc0n...,b'\x81\xe5\xe1M\xde\x81]1\x83\xca\x0b\xe7\x86\...,b'\x80\xf1\xe7U\xdf\x95]+\x80\xd6\x08\xaf{

In [17]:
decrypted_df_sales

,Jan,Feb,Mar,Apr,Mei,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year Total
0,"377,045,713","379,113,097","372,316,090","392,567,778","398,734,898","411,232,840","368,853,507","415,321,833","382,962,945","467,893,711","375,815,944","424,064,387","4,765,922,743"
1,"242,370,197","212,251,985","204,414,196","233,523,016","220,730,925","233,194,586","221,971,536","246,355,021","236,335,091","292,120,864","201,786,738","243,465,095","2,788,519,250"
2,"576,420,037","528,756,483","546,400,555","617,218,714","587,424,126","547,152,791","550,558,762","555,374,056","478,131,369","580,774,960","418,898,225","533,925,422","6,521,035,500"
3,"262,642,225","247,718,521","281,930,873","330,248,572","274,214,556","292,985,200","281,301,030","302,722,271","272,821,443","337,484,741","213,824,938","270,660,844","3,368,555,214"
4,"283,962,745","273,753,242","275,621,256","275,377,925","285,094,002","271,323,533","266,368,767","278,199,918","274,482,338","340,876,987","272,031,107","319,181,859","3,416,273,679"
5,"264,618,010","262,560,291","266,737,078","291,233,128","266,804,240","265,814,874","262,475,102","327,169,088","279,381,522","341,443,345","228,436,701","285,087,305","3,341,760,684"
6,"217,605,126","202,406,935","205,281,943","251,661,179","205,936,466","222,309,023","228,219,727","216,370,586","205,156,152","248,363,183","155,247,937","172,204,443","2,530,762,700"
7,"250,304,747","220,955,617","239,413,809","279,512,633","223,743,139","258,200,971","240,139,882","271,225,530","231,088,815","313,723,510","218,526,485","287,813,635","3,034,648,773"
8,"255,194,364","233,694,343","245,456,148","280,149,467","233,751,924","265,984,894","271,807,000","271,945,498","240,868,948","287,434,527","181,170,109","230,451,812","2,997,909,034"
9,"480,110,487","449,252,634","441,566,370","392,514,687","476,342,526","505,133,298","442,169,670","477,236,876","458,940,797","520,116,162","414,042,405","498,157,979","5,555,583,891"
